In [180]:
import pandas as pd
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [195]:
shops_SPB = pd.read_csv('/home/ruslan/Programming/Python/Hackathon/data/OpenStreetMap/shops.csv').dropna()
shops_SPB.rename(columns={'lat': 'Latitude', 'lon': 'Longitude'}, inplace=True)
shops_SPB.drop('name', inplace = True, axis = 1)
list_of_food_shops = ['supermarket', 'convenience', 'bakery', 'food', 'seafood', 'milk', 'vegetables']
shops_SPB = shops_SPB[shops_SPB['shop'].isin(list_of_food_shops)]

In [204]:
# rest_NY = pd.read_csv('NY_Restaurants.csv').dropna()
amen_SPB = pd.read_csv('/home/ruslan/Programming/Python/Hackathon/data/OpenStreetMap/amenities.csv').dropna()
amen_SPB.rename(columns={'lat': 'Latitude', 'lon': 'Longitude'}, inplace=True)

list_of_rest = ['restaurant', 'cafe', 'fast_food', 'nightclub', 'food_court']
list_of_educ = ['kindergarten', 'university', 'school', 'college', 'language_school', 'sport_school', 'music_school', 'preschool']
list_of_infrastr = ['bank', 'fuel', 'cinema', 'theatre', 'police', 'offices', 'training', 'gym', 'parking']
list_of_med = ['doctors', 'dentist', 'clinic', 'healthcare', 'hospice', 'rescue_station']


rest_SPB = amen_SPB[amen_SPB['amenity'].isin(list_of_rest)]
educ_SPB = amen_SPB[amen_SPB['amenity'].isin(list_of_educ)]
infrastr_SPB = amen_SPB[amen_SPB['amenity'].isin(list_of_infrastr)]
med_SPB = amen_SPB[amen_SPB['amenity'].isin(list_of_med)]


In [218]:
rest_SPB.to_csv('rest_SPB.csv', index = False)
educ_SPB.to_csv('educ_SPB.csv', index = False)
infrastr_SPB.to_csv('infrastr_SPB.csv', index = False)
med_SPB.to_csv('med_SPB.csv', index = False)
shops_SPB.to_csv('shops_SPB.csv', index = False)

In [238]:
# all_SPB = pd.read_csv('olya_features.csv', usecols = [1, 2])
# all_SPB.rename(columns={'lat': 'Latitude', 'lng': 'Longitude'}, inplace=True)
# all_SPB['food_retail'] = 0; all_SPB['rest'] = 0; all_SPB['educ'] = 0; all_SPB['infrastr'] = 0; all_SPB['med'] = 0;
all_SPB.tail()

,Longitude,Latitude,food_retail,rest,educ,infrastr,med
17192,30.499496,59.813234,-1,-1,0,0,-1
17193,30.265992,60.112603,0,-1,0,0,-1
17194,30.240136,59.849990,1,0,0,1,1
17195,30.325358,60.015253,1,1,1,1,2
17196,30.352713,59.949238,2,2,2,2,2


In [240]:
#shops
sets = [shops_SPB, rest_SPB, educ_SPB, infrastr_SPB, med_SPB]
names = ['food_retail', 'rest', 'educ', 'infrastr', 'med']

for j in range(len(sets)):
    all_scores = score_matrix(sets[j])
    for i in range(len(all_SPB)):
        lat = all_SPB.iloc[i, 1]
        long = all_SPB.iloc[i, 0]
        all_SPB.iloc[i, 2 + j] = score(sets[j], lat, long, np.log(all_scores).mean(), np.log(all_scores).std())
    

In [241]:
all_SPB.to_csv('all_spb2.csv', index = False)

In [187]:
def score_matrix(df, r=1):
    one_km = 0.008913
    km = one_km * r
    all_scores = []
    for lat in np.arange(df['Latitude'].min(), df['Latitude'].max(), 2 * km):
        for long in np.arange(df['Longitude'].min(), df['Longitude'].max(), 2 * km):
            tmp = len(df[(df.Latitude < lat + km) & (df.Latitude > lat - km) &
                                          (df.Longitude < long + km) & (df.Longitude > long - km)])
            if tmp != 0:
                all_scores.append(tmp)
    
    return all_scores

In [188]:
def score(df, lat, long, all_mean, all_std,  r=1):
    one_km = 0.008913
    km = one_km * r
    
    result = (np.log(len(df[(df.Latitude < lat + km) & (df.Latitude > lat - km) &
        (df.Longitude < long + km) & (df.Longitude > long - km)]) + 1) - all_mean) / all_std
    
    return result